# Self-Replicating Robot System Configurations

|ID|Design Option|Characteristics|
|-|-|-|
|1|Centralized homogeneous (CHO)|One robot is responsible for both printing components and assembling them. Constructed robots are of the normal type and either gather resources or complete other objectives.|
|2|Decentralized homogeneous (DHO)| All robots have the capability to print components, assemble them, and gather resources or complete other objectives.|
|3|Hierarchical homogeneous (HHO)|There are a variable number of robots capable of printing components and assembling them. There are also a variable number of normal type robots.|
|4|Centralized heterogeneous (CHE)|One robot is responsible for printing components, and another (distinct) robot is responsible for assembling them. Constructed robots are of the normal type and either gather resources or complete other objectives.|
|5|Decentralized heterogeneous (DHE)|Robots have either the capability to print components or the capability to assemble them. All robots can gather resources or complete other objectives.|
|6|Hierarchical heterogeneous (CHE)|There are a variable number of robots capable of printing components, a variable number capable of assembling them (distinct from the printing group), and a variable number of normal type robots. All robots can gather resources or complete other objectives.|



# Libraries 

In [281]:
import platform
print(f'Python version: {platform.python_version()}')
import random
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import sys
from matplotlib.patches import Rectangle
import os
from _plotly_future_ import v4_subplots
import plotly
import plotly.io as pio
import numpy as np
import chart_studio
import chart_studio.plotly as cspy
import chart_studio.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
username = 'vishalgattani' # your username\n",
api_key = 'WSy2EFPTbxYYm3Rmcx53' # your api key - go to profile > settings > regenerate key\n",
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

Python version: 3.9.14


# Global Variables

In [282]:
# set random number generator
random.seed()
# global variables
rid = 1
nid = 0
aid = 0
pid = 0
decimalPlaces = 3
# simulation parameters
mc = False
mc_num_steps = 500
numTasksBeforeFail = 5
timesteps = 100				#; % Number of iterations/time-steps that the simulation goes through.
NonPr = 300.0 				#; % The robot system=s starting quantity of nonprintable components.
Printable = 100.0 			#; % The robot system’s starting quantity of printable components.
Materials = 50.0 			#; % The robot system’s starting quantity of raw printing materials.
Env_Materials = 500.0 		#; % The environment’s quantity of collectable raw printing materials.
BaseCost_NonPr = 1 			#; % Base robot cost of nonprintable components.
PrintCost_NonPr = 1 		#; % Print capability cost of nonprintable components.
AssembleCost_NonPr = 1 		#; % Assemble capability cost of nonprintable components.
BaseCost_Pr = 2 			#; % Base robot cost of printable components.
PrintCost_Pr = 2 			#; % Print capability cost of printable components.
AssembleCost_Pr = 2 		#; % Assemble capability cost of printable components.
BaseCost_Time = 2 			#; % Base robot cost of build time (in time-steps).
PrintCost_Time = 2 			#; % Print capability cost of build time (in time-steps).
AssembleCost_Time = 2 		#; % Assemble capability cost of build time (in time-steps).
Print_Efficiency = 1.0 		#; % Factor that scales raw printing materials to printable components.
Print_Amount = 1.0 			#; % Amount of raw materials converted per print task.
Collect_Amount = 1.0 		#; % Raw printing materials per collecting robot per timestep.
QualityThreshold = 0.5 		#; % Robots with a quality below this are non-functional.
Quality_incr_Chance = 5.0 	#; % Chance that a new robot’s build quality will increase.
Quality_incr_Lower = 0.01 	#; % Lower bound for quality increase amount.
Quality_incr_Upper = 0.05 	#; % Upper bound for quality increase amount.
Quality_decr_Chance = 50.0 	#; % Chance that a new robot s build quality will decrease.
Quality_decr_Lower = 0.01 	#; % Lower bound for quality decrease amount.
Quality_decr_Upper = 0.25 	#; % Upper bound for quality decrease amount.
RiskAmount_Collect = 1.0 	#; % Risk chance for the collect task type.
RiskAmount_Assemble = 0.1 	#; % Risk chance for the assemble task type.
RiskAmount_Print = 0.1 		#; % Risk chance for the print task type.
RiskQuality_Modifier = 1.0 	#; % Multiplier for impact of quality defects on risk amount.
RiskFactory_Modifier = 0.2 	#; % Multiplier for impact of factory-made robots on risk amount
RiskThreshold = 3.0
# [replicator,normal,assembler,printer]
cost_Pr = [6,2,4,4]			#; % Total cost printable
cost_NonPr = [3,1,2,2]		#; % Total cost nonprintable

timecost_base = 2			#; % time cost basic
timecost_print = 2			#; % time cost print capability
timecost_assemble = 2		#; % time cost assemble capability

timecost_normal = timecost_base
timecost_replicator = timecost_base+timecost_assemble+timecost_print
timecost_printer = timecost_base+timecost_print
timecost_assembler = timecost_base+timecost_assemble

timecost_repair_base = 1
timecost_repair_normal = timecost_repair_base
timecost_repair_replicator = timecost_repair_base
timecost_repair_printer = timecost_repair_base
timecost_repair_assembler = timecost_repair_base

table_columns = ["Time","NonPr","Printable","Materials","Env_Materials",
                "#Replicator","#Normal","#Assembler","#Printer",
                "#Assembling","#Printing","#Collecting","#Idle","#Repair",
                "#In","#Out",
                "Average Build Quality in-service","Average Build Quality of System",
                "#WasteReplicator","#WasteNormal","#WasteAssembler","#WastePrinter",
                "Environment Exhaust Time", "Printable Exhaust Time", 
                "NonPr Exhaust Time","Material Exhaust Time","Average Risk"]

def resetGlobal(t,r,n,a,p):
    global rid,nid,aid,pid,Num_Steps,NonPr,Printable,Materials,Env_Materials
    # global variables
    rid = r
    nid = n
    aid = a
    pid = p
    # simulation parameters
    Num_Steps = 100				#; % Number of iterations/time-steps that the simulation goes through.
    NonPr = 300.0 				#; % The robot system=s starting quantity of nonprintable components.
    Printable = 100.0 			#; % The robot system’s starting quantity of printable components.
    Materials = 50.0 			#; % The robot system’s starting quantity of raw printing materials.
    Env_Materials = 500.0 		#; % The environment’s quantity of collectable raw printing materials.
    timesteps = t


# Object::Robot

In [283]:
# robot object
class Robot:
    def __init__(self,type,build_qual,id):
        self.time = 0
        self.type = type
        self.current_task = "idle"
        self.prev_task = "idle"
        self.next_task = ""
        self.id = self.type[0]+str(id)
        self.build_qual = round(build_qual,decimalPlaces)
        self.factory_made = True
        self.tasks_dur = 0
        self.taskindex = 0
        self.previouslyBuilt = ""
        self.prevTaskDur = 0
        self.curr_repair_task_dur = 0
        self.prev_repair_task_dur = 0
        self.factory = False
        self.riskAmount = 0
        self.riskSet = False
        self.numTasksSuccess = 0
        self.numTasksFailed = 0
        self.numTasksPerformed = 0
        self.downTime = 0
        self.failureTimes = [0]
        self.operationalTimes = [0]
        self.operationalTime = 0
        self.MTBF = 0
        self.MTTR = 0
        self.MDT = 0
        self.Aoss = 0
        self.numRepairs = 0
        self.beingBuilt = ""
        self.numTasksBeforeFailure = 2
        self.numTasksRemainingBeforeFailure = 2
        if(self.type == "Replicator"):
            self.tasks = ["Assemble","Print","Collect","Repair"]
            self.beingbuiltlist = []
        if(self.type == "Normal"):
            self.tasks = ["Collect","Repairing"]
        if(self.type == "Assembler"):
            self.tasks = ["Assemble","Collect","Repair"]
            self.beingbuiltlist = []
        if(self.type == "Printer"):
            self.tasks = ["Print","Collect","Repair"]
        self.num_tasks = len(self.tasks)
    # robot methods/functions definitions
    def reduceNumTasksRemainingBeforeFailure(self):
        self.numTasksRemainingBeforeFailure=self.numTasksRemainingBeforeFailure-1
    def resetNumTasksRemainingBeforeFailure(self):
        self.numTasksRemainingBeforeFailure=self.numTasksBeforeFailure
    def setNumTasksBeforeFailure(self,val):
        self.numTasksBeforeFailure = val
    def getNumTasksBeforeFailure(self):
        return self.numTasksBeforeFailure
    def getNumTasksRemainingBeforeFailure(self):
        return self.numTasksRemainingBeforeFailure
    def setNumTasksRemainingBeforeFailure(self,val):
        self.numTasksRemainingBeforeFailure = val
    def set_being_built(self,robottype):
        self.beingBuilt = robottype
    def get_being_built(self):
        return self.beingBuilt
    def set_previously_built(self, val):
        self.previouslyBuilt = val
    def get_previously_built(self):
        return self.previouslyBuilt
    def setNumTasksPerformed(self):
        self.numTasksPerformed = self.numTasksFailed+self.numTasksSuccess
    def getNumTasksPerformed(self):
        return self.numTasksPerformed
    def computeMTBF(self):
        try:
            if(self.operationalTime==0 and self.numTasksFailed>0):
                self.MTBF = np.nan
            else:
                self.MTBF = self.operationalTime/self.numTasksFailed
        except:
            if self.getNumTasksPerformed() == 0:
                self.MTBF = np.nan
            else:
                self.MTBF = self.operationalTime
    def computeMTTR(self):
        try:
            self.MTTR = self.downTime/self.numRepairs
        except:
            if self.getNumTasksPerformed() == 0:
                self.MTTR = np.nan
            elif(self.downTime == 0 or self.numRepairs==0):
                self.MTTR = self.getMTBF()
    def computeMDT(self):
        try:
            self.MDT = self.downTime/self.numTasksFailed
        except:
            if self.getNumTasksPerformed() == 0:
                self.MDT = np.nan
            elif(self.downTime == 0):
                self.MDT = math.inf
    def computeAoss(self):
        try:
            if(self.numTasksFailed == 0 and self.downTime == 0):
                self.Aoss =  self.MTBF/(self.MTBF+0)
            else:
                self.Aoss =  self.MTBF/(self.MTBF+self.MDT)
        except:
            if self.getNumTasksPerformed() == 0:
                self.Aoss = np.nan
    def computeRAM(self):
        self.setNumTasksPerformed()
        self.computeMTBF()
        self.computeMTTR()
        self.computeMDT()
        self.computeAoss()
    def printRAM(self):
        return (f'{self.id} - MTBF:{self.MTBF:.2f}, MTTR:{self.MTTR:.2f}, MDT:{self.MDT:.2f}, Aoss:{self.Aoss}')
    def getMTBF(self):
        return self.MTBF 
    def getMTTR(self):
        return self.MTTR 
    def getMDT(self):
        return self.MDT 
    def getAoss(self):
        return self.Aoss 
    def addNumRepairs(self):
        self.numRepairs += 1
    def taskFail(self):
        self.numTasksFailed += 1
    def taskSuccess(self):
        self.numTasksSuccess += 1
    def gettaskFail(self):
        return self.numTasksFailed
    def gettaskSuccess(self):
        return self.numTasksSuccess
    def get_type(self):
        return self.type
    def __str__(self):
        try:
            return str(f'{self.id},{self.current_task},{self.tasks_dur},tp:{self.gettaskSuccess()}')
        except:
            return None
    def set_curr_task(self,tasktype):
        self.current_task = tasktype
        if(self.current_task == "idle"):
            self.tasks_dur = 0
        if(self.current_task == "collecting"):
            self.tasks_dur = 1
        if(self.current_task == "assembling"):
            self.tasks_dur = 2
        if(self.current_task == "printing"):
            self.tasks_dur = 2
        if(self.current_task == "repair"):
            self.failureTimes.append(self.getRobotTime())
            if(self.prev_task == "repair"):
                self.curr_repair_task_dur = self.get_task_dur()
                self.prev_repair_task_dur = self.curr_repair_task_dur
                self.tasks_dur = self.get_prev_task_dur() + 1
            else:
                self.tasks_dur = self.get_prev_task_dur() 
            self.curr_repair_task_dur = self.tasks_dur
    def addDownTime(self,duration):
        self.downTime += duration
    def getDownTime(self):
        return self.downTime
    def addUpTime(self):
        self.upTime += 1
    def getUpTime(self):
        return self.upTime
    def resetUpTime(self):
        self.upTime = 0
    def addOperationalTime(self,duration):
        self.operationalTime += duration
    def getOperationalTime(self):
        return self.operationalTime
    def setFactory(self):
        self.factory = True
    def setRobotTime(self,time):
        self.time = time
    def getRobotTime(self):
        return self.time
    def set_prev_task(self,tasktype):
        self.prev_task = tasktype
    def set_next_task(self,tasktype):
        self.next_task = tasktype
    def set_task_dur(self,task_dur):
        self.tasks_dur = task_dur
    def get_task_dur(self):
        return self.tasks_dur
    def get_robot_id(self):
        return self.id
    def get_buid_qual(self):
        return self.build_qual
    def get_curr_task(self):
        return self.current_task
    def get_prev_task(self):
        return self.prev_task
    def get_next_task(self):
        return self.next_task
    def get_task_dur(self):
        return self.tasks_dur
    def get_prev_task_dur(self):
        if(self.get_prev_task() == "idle"):
            return 0
        if(self.get_prev_task() == "collecting"):
            return 1
        if(self.get_prev_task() == "assembling"):
            return 2
        if(self.get_prev_task() == "printing"):
            return 2
        if(self.get_prev_task() == "repair"):
            return self.prev_repair_task_dur 
    def setRiskAmount(self,risk):
        self.riskAmount = risk
        if risk>0.0:
            self.riskSet = True
        else:
            self.riskSet = False
    def getRiskAmount(self):
        return self.riskAmount
    def getRisk(self):
        return self.riskSet


# Function::Configuration Handler

In [284]:
def configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat):
        n_replicator = 0
        n_normal = 0
        n_assembler = 0
        n_printer = 0

        c_flag = 0
        p_flag = 0
        a_flag = 0
        i_flag = 0
        repair_flag = 0

        useless_c_flag = 0
        useless_p_flag = 0
        useless_a_flag = 0
        useless_r_flag = 0

        tot_build_qual_inservice = 0
        tot_build_qual_inoutservice = 0

        build_quality_list = []

        for i in totlist:
            if(i.get_buid_qual()>=0.5):
                tot_build_qual_inoutservice = tot_build_qual_inoutservice + i.get_buid_qual()
                tot_build_qual_inservice = tot_build_qual_inservice + i.get_buid_qual()
            else:
                tot_build_qual_inoutservice = tot_build_qual_inoutservice + i.get_buid_qual()
            build_quality_list.append(i.get_buid_qual())
        
        avg_risk = 0
        for i in robotlist:
            if i.get_curr_task()=="collecting":
                c_flag+=1
            elif i.get_curr_task()=="printing":
                p_flag+=1
            elif i.get_curr_task()=="assembling":
                a_flag+=1
            elif i.get_curr_task()=="idle":
                # print(i)
                i_flag+=1
            elif i.get_curr_task()=="repair":
                # print(i)
                repair_flag+=1

            if(checkType(i,"Replicator")):
                n_replicator += 1;
            elif(checkType(i, "Normal")):
                n_normal += 1;
            elif(checkType(i, "Assembler")):
                n_assembler += 1;
            elif(checkType(i, "Printer")):
                n_printer += 1;
                
            avg_risk += i.getRisk()
        avg_risk = avg_risk/len(robotlist)
        
        for i in useless:
            if(checkType(i,"Replicator")):
                useless_r_flag += 1;
            elif(checkType(i, "Normal")):
                useless_c_flag += 1;
            elif(checkType(i, "Assembler")):
                useless_a_flag += 1;
            elif(checkType(i, "Printer")):
                useless_p_flag += 1;



        avg_build_qual_inservice = round(tot_build_qual_inservice/len(robotlist),decimalPlaces)
        avg_build_qual_inoutservice = round(tot_build_qual_inoutservice/len(totlist),decimalPlaces)
        
        neatPrint = False
        if neatPrint:
            print("="*50)
            print(t,":\t\t",len(robotlist),[NonPr,Printable,Materials,Env_Materials])			
            print("Time\t\t",t)
            print("#Replicator:\t",n_replicator)
            print("#Normal:\t",n_normal)
            print("#Assembling:\t",n_assembler)
            print("#Printing:\t",n_printer)
            print("#Robots\t\t",len(robotlist))
            print("Materials\t",[NonPr,Printable,Materials,Env_Materials])
            print("#Assembling:\t",a_flag)
            print("#Printing:\t",p_flag)
            print("#Collecting:\t",c_flag)
            print("#Idle:\t\t",i_flag)
            print("#Repair:\t\t",repair_flag)
            
        ids=[]
        for j in totlist:
            isWaste = False
            if j.build_qual<=QualityThreshold:
                isWaste = True
            ids.append(j.id)

        if (Env_Materials == 0 and checkENV == 0):
            checkENV = t
        if (Printable == 0 and checkPrint == 0):
            checkPrint = t
        if (NonPr == 0 and checkNonPr == 0):
            checkNonPr = t
        if (Materials <= 1 and checkMat == 0):
            checkMat = t
            
        return build_quality_list,[n_replicator,n_normal,n_assembler,n_printer,
        a_flag,p_flag,c_flag,i_flag,repair_flag,
        len(robotlist),len(useless),
        avg_build_qual_inservice,avg_build_qual_inoutservice,
        useless_r_flag,useless_c_flag,useless_a_flag,useless_p_flag,
        checkENV,checkPrint,checkNonPr,checkMat,avg_risk]

# Functions::Tasks

In [285]:
# print current resources
def printResources():
    print(NonPr,Printable,Materials,Env_Materials)

# check if robot can collect from Env_Materials
def collectCheck(robot):
    global Materials, Env_Materials, Collect_Amount
    if (Env_Materials - Collect_Amount >= 0):
        robot.setRiskAmount(taskRisk(robot))
        return True
    else:
        robot.setRiskAmount(0)
        return False

# task function - collecting
def collecting(robot):
    global Materials, Env_Materials, Collect_Amount, RiskThreshold,mc,numTasksBeforeFail
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("collecting")
    robot.set_task_dur(1)
    flag = 1
    if not mc:
        if robot.getNumTasksRemainingBeforeFailure()==0:
            robot.taskFail()
            flag = 0
            repairing(robot)     
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            Materials = Materials + Collect_Amount
            Env_Materials = Env_Materials - Collect_Amount
            robot.taskSuccess()
            robot.reduceNumTasksRemainingBeforeFailure()
    if mc:
        if robot.getRisk() == False:
            robot.setRiskAmount(taskRisk(robot))
        elif robot.getRiskAmount() > RiskThreshold:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            Materials = Materials + Collect_Amount
            Env_Materials = Env_Materials - Collect_Amount
            robot.taskSuccess()
    
def assembleCheck(robot,tobuild):
    if(tobuild == "Replicator"):
        i=0
    if(tobuild == "Normal"):
        i=1
    if(tobuild == "Assembler"):
        i=2
    if(tobuild == "Printer"):
        i=3
    if Printable - cost_Pr[i] >= 0 and NonPr - cost_NonPr[i] >= 0:
        if(robot.getRisk() == False):
            robot.setRiskAmount(taskRisk(robot))
        return True
    else:
        robot.setRiskAmount(0)
        return False


def assembling(robot,tobuild):
    global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper,RiskThreshold,mc,numTasksBeforeFail
    if(tobuild == "Replicator"):
        i=0
        taskDur = timecost_replicator
    if(tobuild == "Normal"):
        i=1
        taskDur = timecost_normal
    if(tobuild == "Assembler"):
        i=2
        taskDur = timecost_assembler
    if(tobuild == "Printer"):
        i=3
        taskDur = timecost_printer
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("assembling")
    robot.set_task_dur(taskDur)
    flag = 1
    if not mc:
        if robot.getNumTasksRemainingBeforeFailure()==0:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            if(robot.type=="Assembler" or robot.type=="Replicator"):
                if(tobuild == "Replicator"):
                    i=0
                    rid = rid+1
                    robotid = rid
                if(tobuild == "Normal"):
                    i=1
                    nid = nid+1
                    robotid = nid
                if(tobuild == "Assembler"):
                    i=2
                    aid = aid+1
                    robotid = aid
                if(tobuild == "Printer"):
                    i=3
                    pid = pid+1
                    robotid = pid
                # subtract resources
                Printable = Printable - cost_Pr[i]
                NonPr = NonPr - cost_NonPr[i]
                robot.beingbuiltlist.append(tobuild[0]+str(robotid))
                robot.set_previously_built(tobuild)
                robot.reduceNumTasksRemainingBeforeFailure()
                return True
        else:
            return False
    if mc:
        if robot.getRisk() == True and robot.get_prev_task()=="assembling":
            robot.setRiskAmount(taskRisk(robot))
        if robot.getRiskAmount() > RiskThreshold:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            if(robot.getRisk() == True and robot.get_task_dur() - 1 == 0):
                robot.setRiskAmount(0)

            if(robot.type=="Assembler" or robot.type=="Replicator"):
                if(tobuild == "Replicator"):
                    i=0
                    rid = rid+1
                    robotid = rid
                if(tobuild == "Normal"):
                    i=1
                    nid = nid+1
                    robotid = nid
                if(tobuild == "Assembler"):
                    i=2
                    aid = aid+1
                    robotid = aid
                if(tobuild == "Printer"):
                    i=3
                    pid = pid+1
                    robotid = pid
                # subtract resources
                Printable = Printable - cost_Pr[i]
                NonPr = NonPr - cost_NonPr[i]
                robot.beingbuiltlist.append(tobuild[0]+str(robotid))
                robot.set_previously_built(tobuild)
                return True
        else:
            robot.set_prev_task(robot.get_curr_task())
            robot.set_curr_task("idle")
            robot.set_task_dur(0)
            robot.setRiskAmount(0)
            return False

def assemble(builder,tobuild):
    global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper, mc
    if(builder.type=="Assembler" or builder.type=="Replicator"):
        if(tobuild == "Replicator"):
            i=0
        if(tobuild == "Normal"):
            i=1
        if(tobuild == "Assembler"):
            i=2
        if(tobuild == "Printer"):
            i=3
        AssemblerQuality = builder.get_buid_qual()
        # robot's build quality
        if not mc:
            RobotQuality = AssemblerQuality
        if mc:
            rand = round(random.uniform(0,1),decimalPlaces)
            if rand > round((1.0 - Quality_incr_Chance/100),decimalPlaces):
                RobotQuality = AssemblerQuality + random.uniform(Quality_incr_Lower, Quality_incr_Upper)
            elif rand < Quality_decr_Chance :
                RobotQuality = AssemblerQuality - random.uniform(Quality_decr_Lower, Quality_decr_Upper)
            else :
                RobotQuality = AssemblerQuality
        newRobot = Robot(tobuild,RobotQuality,builder.beingbuiltlist.pop(0)[1:])
        builder.taskSuccess()
        return newRobot
    else:
        return None


def printCheck(robot):
    if(robot.type=="Replicator" or robot.type=="Printer"):
        global Print_Efficiency, Print_Amount, Materials, Printable
        if Materials - (Print_Efficiency*Print_Amount) > 0:
            return True
        else:
            return False
    else:
        return False

def printing(robot):
    global Print_Efficiency, Print_Amount, Materials, Printable,mc,numTasksBeforeFail
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("printing")
    robot.set_task_dur(PrintCost_Time)
    flag = 1
    if not mc:
        if robot.getNumTasksRemainingBeforeFailure()==0:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            robot.taskSuccess()
            Materials = Materials - (Print_Efficiency*Print_Amount)
            Printable = Printable + (Print_Efficiency*Print_Amount)
            robot.reduceNumTasksRemainingBeforeFailure()
    if mc:
        if robot.getRisk() == False:
            robot.setRiskAmount(taskRisk(robot))
        if robot.getRiskAmount() > RiskThreshold:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            if(robot.getRisk() == True and robot.get_task_dur() - 1 == 0):
                robot.taskSuccess()
                robot.setRiskAmount(0)
            Materials = Materials - (Print_Efficiency*Print_Amount)
            Printable = Printable + (Print_Efficiency*Print_Amount)

def repairing(robot):
    global mc
    robot.set_next_task(robot.get_curr_task())
    robot.set_prev_task(robot.get_curr_task())
    repair_task_dur = robot.get_task_dur() # + basecost to repair? 
    robot.set_curr_task("repair")
    robot.addNumRepairs()
    robot.set_task_dur(repair_task_dur)
    robot.addDownTime(repair_task_dur)
    if not mc:
        robot.resetNumTasksRemainingBeforeFailure()

def resetTasks(robot):
    robot.set_prev_task(robot.get_curr_task())
    robot.set_task_dur(0)
    robot.set_curr_task("idle")

def checkCurrentTask(robot,current_task):
    return robot.get_curr_task() == current_task
def checkPreviousTask(robot,previous_task):
    return robot.get_prev_task() == previous_task

def checkType(robot,robot_type):
    return robot.get_type() == robot_type



# Task Risk

RiskTask = RiskTask_Type + (1.0 − RobotQuality) ∗ RiskTask_Type ∗ RiskQuality_Modifier

if Robot ∈ FactoryMade : RiskTask = RiskTask ∗ RiskFactory_Modifier

if robot is factory made
riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskFactory_Modifier)

if robot is not factor made
riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskQuality_Modifier)

In [359]:
def taskRisk(robot):
    rand = round(random.uniform(0,1),decimalPlaces)
    if robot.factory == True:
        currTask = robot.get_curr_task()
        if(currTask == "idle"):
            RiskTask_Type = 0
        elif(currTask == "collecting"):
            RiskTask_Type = 1
        elif(currTask == "assembling"):
            RiskTask_Type = 1
        elif(currTask == "printing"):
            RiskTask_Type = 1
        elif(currTask == "repair"):
            RiskTask_Type = 0
        riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskFactory_Modifier)
#         riskTask = (1.0 - rand) * (RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskFactory_Modifier)
    else:
        currTask = robot.get_curr_task()
        if(currTask == "idle"):
            RiskTask_Type = 0            
        elif(currTask == "collecting"):
            RiskTask_Type = 1
        elif(currTask == "assembling"):
            RiskTask_Type = 1
        elif(currTask == "printing"):
            RiskTask_Type = 1
        elif(currTask == "repair"):
            RiskTask_Type = 0
        riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskQuality_Modifier)
#         riskTask = (1.0 - rand) * (RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskQuality_Modifier)
    return riskTask

# CHO

In [360]:
def CHO(timesteps,df,init_build_qual,printRAMmetrics=True,mc=False):
    global numTasksBeforeFail
    resetGlobal(timesteps,1,0,0,0)
    robot = Robot("Replicator",init_build_qual,rid)
    robot.setFactory()
    totlist = [robot]
    robotlist = [robot]
    useless = []
    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []
    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0

    for t in range(0,timesteps):
        for i in range(len(robotlist)):
            # assign risks for tasks
            robotlist[i].setRobotTime(t)
            robotlist[i].setNumTasksBeforeFailure(numTasksBeforeFail)
            # IDLE
            if(robotlist[i].get_curr_task()=="idle"):
                # Replicator
                if(robotlist[i].type == "Replicator"):
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])	
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Normal
                elif(robotlist[i].type == "Normal"):
                    if (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
            # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                nextTask = robotlist[i].get_next_task()
                # tasks of replicator in CHO
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Replicator"):
                    if(nextTask=="assembling" and assembleCheck(robotlist[i],"Normal")):
                        assembling(robotlist[i],"Normal")
                    elif(nextTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of normal in CHO
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Normal"):
                    if(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                    # it enters this loop only when it has to pop a new robot
                    if(len(robotlist[i].beingbuiltlist)>0 and robotlist[i].get_prev_task()=="assembling"):
                        newbot = assemble(robotlist[i],"Normal")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if(newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)
                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if(canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
            
#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)
            
        #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)

    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    df["#Tasks before failure"] = robotlist[0].getNumTasksBeforeFailure()
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# DHO

In [361]:
def DHO(timesteps,df,init_build_qual):
    resetGlobal(timesteps,1,0,0,0)
    robot = Robot("Replicator",init_build_qual,rid)
    robot.setFactory()
    totlist = [robot]
    robotlist = [robot]
    useless = []
    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []
    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    for t in range(0,timesteps):
        for i in range(len(robotlist)):
            robotlist[i].setRobotTime(t)
            robotlist[i].setNumTasksBeforeFailure(numTasksBeforeFail)
            # IDLE
            if(robotlist[i].current_task=="idle"):

                # Replicator
                if(robotlist[i].type == "Replicator"):
                    if(assembleCheck(robotlist[i],"Replicator")):
                        assembling(robotlist[i],"Replicator")
                    elif(printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    # print(t,robotlist[i].id,canCollect)
                    if canCollect:
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
         # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                nextTask = robotlist[i].get_next_task()
                # tasks of replicator in DHO
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Replicator"):
                    if(nextTask=="assembling" and assembleCheck(robotlist[i],"Replicator")):
                        assembling(robotlist[i],"Replicator")
                    elif(nextTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                            
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Replicator")):
                        isAssembling = assembling(robotlist[i],"Replicator")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    if(len(robotlist[i].beingbuiltlist)>0 and robotlist[i].get_prev_task()=="assembling"):
                        newbot = assemble(robotlist[i],"Replicator")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if(newbot.type == "Replicator"):
                                if(assembleCheck(newbot,"Replicator")):
                                    assembling(newbot,"Replicator")
                                elif(printCheck(robotlist[i])):
                                    printing(robotlist[i])
                                elif(collectCheck(robotlist[i])):
                                    collecting(robotlist[i])
                                else:
                                    newbot.set_prev_task(robotlist[i].get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)

#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)

         #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)

    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    df["#Tasks before failure"] = robotlist[0].getNumTasksBeforeFailure()
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# HHO

In [362]:
def HHO(timesteps,df,init_build_qual):
    resetGlobal(timesteps,1,0,0,0)
    robot = Robot("Replicator",init_build_qual,rid)
    robot.setFactory()
    totlist = [robot]
    robotlist = [robot]
    useless = []
    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []
    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0,timesteps):
        for i in range(len(robotlist)):
            robotlist[i].setRobotTime(t)
            robotlist[i].setNumTasksBeforeFailure(numTasksBeforeFail)
            # IDLE
            if(robotlist[i].current_task=="idle"):
                # Replicator
                if(robotlist[i].type == "Replicator"):
                    if (robotlist[i].get_previously_built() == ""):
                        if (assembleCheck(robotlist[i],"Replicator")):
                            isAssembling = assembling(robotlist[i], "Replicator")
                        elif (assembleCheck(robotlist[i],"Normal")):
                            isAssembling = assembling(robotlist[i], "Normal")
                        elif (printCheck(robotlist[i])):
                            printing(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")
                    else:
                        if (assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Replicator"):
                            isAssembling=assembling(robotlist[i], "Normal")
                        elif (assembleCheck(robotlist[i],"Replicator") and robotlist[i].get_previously_built() == "Normal"):
                            isAssembling=assembling(robotlist[i], "Replicator")
                        elif (printCheck(robotlist[i])):
                            printing(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")

                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    # print(t,robotlist[i].id,canCollect)
                    if canCollect:
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
         # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                nextTask = robotlist[i].get_next_task()
                # tasks of replicator in HHO
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Replicator"):
                    if(nextTask=="assembling" and assembleCheck(robotlist[i],"Replicator") and robotlist[i].get_previously_built() == "Normal"):
                        assembling(robotlist[i],"Replicator")
                    elif(nextTask=="assembling" and assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Replicator"):
                        assembling(robotlist[i],"Normal")
                    elif(nextTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of normal in HHO
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Normal"):
                    if(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                            
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                # Normal
                elif (robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if (canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                        
                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if (assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Replicator"):
                        isAssembling=assembling(robotlist[i], "Normal")
                    elif (assembleCheck(robotlist[i],"Replicator") and robotlist[i].get_previously_built() == "Normal"):
                        isAssembling=assembling(robotlist[i], "Replicator")
                    elif (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    if(len(robotlist[i].beingbuiltlist)>0 and robotlist[i].get_prev_task()=="assembling"):
                        if (robotlist[i].get_previously_built() == "Normal"):
                            newbot = assemble(robotlist[i], "Replicator")
                        elif (robotlist[i].get_previously_built() == "Replicator"):
                            newbot = assemble(robotlist[i], "Normal")
                        else:
                            newbot = assemble(robotlist[i], "Replicator")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if (newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            if(newbot.type == "Replicator"):
                                if (newbot.get_previously_built() == ""):
                                    if (assembleCheck(robotlist[i],"Replicator")):
                                        isAssembling = assembling(newbot, "Replicator")
                                    elif (assembleCheck(robotlist[i],"Normal")):
                                        isAssembling = assembling(newbot, "Normal")
                                    elif (printCheck(newbot)):
                                        printing(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                                else:
                                    if (assembleCheck(robotlist[i],"Normal") and newbot.get_previously_built() == "Replicator"):
                                        assembling(newbot, "Normal")
                                    elif (assembleCheck(robotlist[i],"Replicator") and newbot.get_previously_built() == "Normal"):
                                        assembling(newbot, "Replicator")
                                    # checking if robot can collect
                                    elif (printCheck(newbot)):
                                        printing(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)
                    

#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)

         #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)

    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    df["#Tasks before failure"] = robotlist[0].getNumTasksBeforeFailure()
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# CHE

In [363]:
def CHE(timesteps,df,init_build_qual,printRAMmetrics=True,mc=False):
    resetGlobal(timesteps,0,0,1,1)
    robot1 = Robot("Printer",init_build_qual,pid)
    robot2 = Robot("Assembler",init_build_qual,aid)
    totlist = [robot1,robot2]
    robotlist = [robot1,robot2]
    useless = []
    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []
    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0

    for t in range(0,timesteps):
        for i in range(len(robotlist)):
            robotlist[i].setRobotTime(t)
            robotlist[i].setNumTasksBeforeFailure(numTasksBeforeFail)
            # if idle
            if(robotlist[i].get_curr_task()=="idle"):
                # If idle and Replicator
                if(robotlist[i].type == "Replicator"):
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])	
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # if idle and Normal
                elif(robotlist[i].type == "Normal"):
                    if (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # if idle and printer
                elif(robotlist[i].type == "Printer"):
                    if(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # if idle and assembler
                elif(robotlist[i].type == "Assembler"):
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                
                
            # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                nextTask = robotlist[i].get_next_task()
                # tasks of assembler in CHE
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Assembler"):
                    if(nextTask=="assembling" and assembleCheck(robotlist[i],"Normal")):
                        assembling(robotlist[i],"Normal")
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of printer in CHE
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Printer"):
                    if(nextTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of normal in CHE
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Normal"):
                    if(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                    
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                # Assembler 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Assembler"):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                    # it enters this loop only when it has to pop a new robot
                    if(len(robotlist[i].beingbuiltlist)>0 and robotlist[i].get_prev_task()=="assembling"):
                        newbot = assemble(robotlist[i],"Normal")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if(newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)
                # Printer
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Printer"):
                    if(printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if(canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
              
            
#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)
            
        #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)

    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    df["#Tasks before failure"] = robotlist[0].getNumTasksBeforeFailure()
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# DHE

In [364]:
def DHE(timesteps,df,init_build_qual):
    resetGlobal(timesteps,0,0,1,1)
    robot1 = Robot("Printer",init_build_qual,pid)
    robot2 = Robot("Assembler",init_build_qual,aid)
    totlist = [robot1,robot2]
    robotlist = [robot1,robot2]
    useless = []
    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []
    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    for t in range(0,timesteps):
        for i in range(len(robotlist)):
            robotlist[i].setRobotTime(t)
            robotlist[i].setNumTasksBeforeFailure(numTasksBeforeFail)
            # IDLE
            if(robotlist[i].current_task=="idle"):
                # if idle and printer
                if (robotlist[i].type == "Printer"):
                    if (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # if idle and assembler
                elif (robotlist[i].type == "Assembler"):
                    if (robotlist[i].get_previously_built() == ""):
                        if (assembleCheck(robotlist[i],"Assembler")):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (assembleCheck(robotlist[i],"Printer")):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")
                    else:
                        if (assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Printer"):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")
            # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                nextTask = robotlist[i].get_next_task()
                # tasks of assembler in DHE
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Assembler"):
                    if(nextTask=="assembling" and assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Printer"):
                        assembling(robotlist[i],"Assembler")
                    elif(nextTask=="assembling" and assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                        assembling(robotlist[i],"Printer")
                    elif(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of printer in DHE
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Printer"):
                    if(nextTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                # Printer 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Printer"):
                    if(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Assembler 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Assembler"):
                    # check if it can keep assembling next time step
                    if (assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Printer"):
                        isAssembling=assembling(robotlist[i], "Assembler")
                    elif (assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                        isAssembling=assembling(robotlist[i], "Printer")
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                    # it enters this loop only when it has to pop a new robot
                    if(len(robotlist[i].beingbuiltlist)>0 and robotlist[i].get_prev_task()=="assembling"):
                        if (robotlist[i].get_previously_built() == "Printer"):
                            newbot = assemble(robotlist[i], "Assembler")
                        elif (robotlist[i].get_previously_built() == "Assembler"):
                            newbot = assemble(robotlist[i], "Printer")
                        else:
                            newbot = assemble(robotlist[i], "Assembler")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if(newbot.type == "Assembler"):
                                if (newbot.get_previously_built() == ""):
                                    if (assembleCheck(robotlist[i],"Assembler")):
                                        isAssembling = assembling(newbot, "Assembler")
                                    elif (assembleCheck(robotlist[i],"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    # checking if robot can collect
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                                else:
                                    if (assembleCheck(robotlist[i],"Printer") and newbot.get_previously_built() == "Assembler"):
                                        assembling(newbot, "Printer")
                                    elif (assembleCheck(robotlist[i],"Assembler") and newbot.get_previously_built() == "Printer"):
                                        assembling(newbot, "Assembler")
                                    # checking if robot can collect
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)

#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)

         #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)

    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    df["#Tasks before failure"] = robotlist[0].getNumTasksBeforeFailure()
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# HHE

In [365]:
def HHE(timesteps,df,init_build_qual):
    resetGlobal(timesteps,0,0,1,1)
    robot1 = Robot("Printer",init_build_qual,pid)
    robot2 = Robot("Assembler",init_build_qual,aid)
    totlist = [robot1,robot2]
    robotlist = [robot1,robot2]
    useless = []
    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []
    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0

    for t in range(0,timesteps):
        for i in range(len(robotlist)):
            robotlist[i].setRobotTime(t)
            robotlist[i].setNumTasksBeforeFailure(numTasksBeforeFail)
            # IDLE
            if(robotlist[i].current_task=="idle"):
                # If idle and printer
                if (robotlist[i].type == "Printer"):
                    if (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # If idle and assembler
                if (robotlist[i].type == "Assembler"):
                    if (robotlist[i].get_previously_built() == ""):
                        if (assembleCheck(robotlist[i],"Normal")):
                            isAssembling = assembling(robotlist[i], "Normal")
                        elif (assembleCheck(robotlist[i],"Assembler")):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (assembleCheck(robotlist[i],"Printer")):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")
                    else:
                        if (robotlist[i].get_previously_built() == "Assembler" and assembleCheck(robotlist[i],"Printer")):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (robotlist[i].get_previously_built() == "Printer" and assembleCheck(robotlist[i],"Normal")):
                            isAssembling = assembling(robotlist[i], "Normal")
                        elif (robotlist[i].get_previously_built() == "Normal" and assembleCheck(robotlist[i],"Assembler")):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")

                # If idle and collector
                if (robotlist[i].type == "Normal"):
                    if (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
         # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                nextTask = robotlist[i].get_next_task()
                # tasks of assembler in HHE
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Assembler"):
                    if(nextTask=="assembling" and assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Printer"):
                        assembling(robotlist[i],"Assembler")
                    elif(nextTask=="assembling" and assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Normal"):
                        assembling(robotlist[i],"Printer")
                    elif(nextTask=="assembling" and assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Assembler"):
                        assembling(robotlist[i],"Normal")
                    elif(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of printer in HHE
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Printer"):
                    if(nextTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Normal"):
                    if(nextTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                            
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                # Normal
                elif (robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if (canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Printer 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Printer"):
                    if(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Assembler"):
                    if (assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Printer"):
                        assembling(robotlist[i], "Normal")
                    elif (assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                        assembling(robotlist[i], "Printer")
                    elif (assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Normal"):
                        assembling(robotlist[i], "Assembler")
                    elif (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                        
                    if(len(robotlist[i].beingbuiltlist)>0 and robotlist[i].get_prev_task()=="assembling"):
                        # Build a new robot in a fixed pattern
                        if (robotlist[i].get_previously_built() == ""):
                            newbot = assemble(robotlist[i], "Normal")
                        elif (robotlist[i].get_previously_built() == "Normal"):
                            newbot = assemble(robotlist[i], "Printer")
                        elif (robotlist[i].get_previously_built() == "Printer"):
                            newbot = assemble(robotlist[i], "Assembler")
                        elif (robotlist[i].get_previously_built() == "Assembler"):
                            newbot = assemble(robotlist[i], "Normal")
                        else:
                            newbot = assemble(robotlist[i], "Normal")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if (newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            elif (newbot.type == "Printer"):
                                if (printCheck(newbot)):
                                    printing(newbot)
                                elif (collectCheck(newbot)):
                                    collecting(newbot)
                                else:
                                    newbot.set_prev_task(newbot.get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")
                            elif(newbot.type == "Assembler"):
                                if (newbot.get_previously_built() == ""):
                                    if (assembleCheck(newbot,"Normal")):
                                        isAssembling = assembling(newbot, "Normal")
                                    elif (assembleCheck(newbot,"Assembler")):
                                        isAssembling = assembling(newbot, "Assembler")
                                    elif (assembleCheck(newbot,"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                                else:
                                    if (newbot[i].get_previously_built() == "Assembler" and assembleCheck("Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (newbot[i].get_previously_built() == "Printer" and assembleCheck("Normal")):
                                        isAssembling = assembling(newbot, "Normal")
                                    elif (newbot[i].get_previously_built() == "Normal" and assembleCheck("Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)
                    
#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)

         #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)
    
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    df["#Tasks before failure"] = robotlist[0].getNumTasksBeforeFailure()
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# Testing

In [366]:
def SRRS(config,timesteps,plot=False):
    df = pd.DataFrame(columns = table_columns)
    build_qual_range = [0.85,0.95]
    init_build_qual = random.uniform(build_qual_range[0],build_qual_range[1])
    if config=="CHO":
        df,ram_df = CHO(timesteps,df,init_build_qual)
    elif config=="DHO":
        df,ram_df = DHO(timesteps,df,init_build_qual)
    elif config=="HHO":
        df,ram_df = HHO(timesteps,df,init_build_qual)
    elif config=="CHE":
        df,ram_df = CHE(timesteps,df,init_build_qual)
    elif config=="DHE":
        df,ram_df = DHE(timesteps,df,init_build_qual)
    elif config=="HHE":
        df,ram_df = HHE(timesteps,df,init_build_qual)
        
    if(plot):
        graphOutputs(config,df)
    return df,ram_df

def SRRSmc(config,timesteps,mc_num_steps,plot=True):
    mcSimCols = ['Time', 'NonPr', 'Printable', 'Materials', 'Env_Materials',
           '#Replicator', '#Normal', '#Assembler', '#Printer', '#Assembling',
           '#Printing', '#Collecting', '#Idle', '#Repair', '#In', '#Out',
           'Average Build Quality in-service', 'Average Build Quality of System',
           '#WasteReplicator', '#WasteNormal', '#WasteAssembler', '#WastePrinter',
           'Environment Exhaust Time', 'Printable Exhaust Time',
           'NonPr Exhaust Time', 'Material Exhaust Time', 'Average Risk',
           'Print Capacity', 'Assembling Capacity', 'Collection Capacity', 
           'MTBF','MTTR', 'MDT', 'Aoss']
    
    mcdfsim = pd.DataFrame(columns=mcSimCols)
    for i in range(1,mc_num_steps+1):
        sys.stdout.write("\rRunning iteration: {0}".format(i))
        sys.stdout.flush()
        df,ram_df = SRRS(config,timesteps,plot)
        df_stats = df.describe()
        avg_avgRisk = df_stats.iloc[1]["Average Risk"]
        last_row = df.tail(1)
        last_row["Average Risk"].values[:] = avg_avgRisk
        cols = df.columns
        mcdf = pd.DataFrame(columns=cols)
        mcdf = pd.concat([mcdf,last_row], ignore_index=True)
        mcdf = pd.concat([mcdf, ram_df], axis=1, join='inner')
        last_row = mcdf.tail(1)
        mcdfsim = pd.concat([mcdfsim, last_row], ignore_index=True)
    return mcdfsim
    
def printSRRSConfigRAMmetrics(ram_df):
    display(ram_df)

def graph_RobotTasks(config,df):
    fig = px.bar(df, x="Time", y=['#Printing','#Assembling','#Collecting', '#Idle', '#Repair'], title=config+" Robot Tasks vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show()

def graph_InOutServiceRobots(config,df):
    fig = px.bar(df, x="Time", y=['#In','#Out'], title=config+" Robots in service vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show() 
    
def graph_RobotTypes(config,df):
    fig = px.bar(df, x="Time", y=['#Replicator', '#Normal', '#Assembler', '#Printer'], title=config+" Robots Types vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show() 
    
def graph_Resources(config,df):
#     fig = px.bar(df, x="Time", y=['NonPr', 'Printable', 'Materials', 'Env_Materials'], title="Resources vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["Time"], y=df['NonPr'], mode="lines",name='NonPr'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Printable'], mode="lines",name='Printable'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Materials'], mode="lines",name='Materials'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Env_Materials'], mode="lines",name='Env_Materials'))
    fig.update_layout(hovermode="x",title=config+" Resources vs Time")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Resources')
    fig.show()
    
def graph_AvgBuildQuality(config,df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Average Build Quality in-service'], mode="lines",name='Avg. Build Quality-In Service'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Average Build Quality of System'], mode="lines",name='Avg. Build Quality-System'))
    fig.update_layout(hovermode="x",title=config+" Robot Build Quality vs Time")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Build Quality')
    fig.show()
    
def graphOutputs(config,df):
    graph_RobotTasks(config,df)
    graph_InOutServiceRobots(config,df)
    graph_RobotTypes(config,df)
    graph_Resources(config,df)
    graph_AvgBuildQuality(config,df)
    


    

- [x] CHO 
    - [x] Determinisitc
    - [x] MC
- [X] DHO 
    - [x] Determinisitc
    - [x] MC
- [X] HHO 
    - [X] Determinisitc
    - [X] MC
- [X] CHE 
    - [X] Determinisitc
    - [X] MC
- [X] DHE 
    - [X] Determinisitc
    - [X] MC
- [X] HHE 
    - [X] Determinisitc
    - [X] MC


In [ ]:
def plotDistribution(config,mcdf):
#     ['Average Build Quality in-service', 'Average Build Quality of System',
#     '#Replicator', '#Normal', '#Assembler', '#Printer',
#     'Print Capacity', 'Assembling Capacity', 'Collection Capacity',
#     'MTBF','MTTR', 'MDT', 'Aoss']
#     fig = go.Figure()
#     fig.add_trace(px.histogram(mcdf, x="Aoss", y="MTBF", marginal="rug",
#                    hover_data=['MTBF','MTTR', 'MDT', 'Aoss']))
# #     fig.add_trace(go.Histogram(x=mcdf['Aoss'],hovertemplate="<b>Bin Edges:</b> %{x}<br><b>Count:</b> %{y}<extra></extra>"))
    fig = make_subplots(rows=3, cols=2,subplot_titles=('MTBF','MTTR', 'MDT', 'Aoss','Average Build Quality in-service', 'Average Build Quality of System'))
    trace0 = go.Histogram(x=mcdf['MTBF'])
    trace1 = go.Histogram(x=mcdf['MTTR'])
    trace2 = go.Histogram(x=mcdf['MDT'])
    trace3 = go.Histogram(x=mcdf['Aoss'])
    trace4 = go.Histogram(x=mcdf['Average Build Quality in-service'])
    trace5 = go.Histogram(x=mcdf['Average Build Quality of System'])
    fig.append_trace(trace0, 1, 1)
    fig.append_trace(trace1, 1, 2)
    fig.append_trace(trace2, 2, 1)
    fig.append_trace(trace3, 2, 2)
    fig.append_trace(trace4, 3, 1)
    fig.append_trace(trace5, 3, 2)
    fig.update_layout(hovermode="x",title=config,margin=dict(t=30))
    fig.update_xaxes(title_text='Values')
    fig.update_yaxes(title_text='Count')
    fig.show()
    display(mcdf)
    
def main():
    global RiskThreshold,QualityThreshold,mc_num_steps,timesteps,mc,numTasksBeforeFail
    numTasksBeforeFaillist = [2]
    RiskThresholds=[0.5,0.75]
    timestepslist=[500]
    
#     numTasksBeforeFaillist = [2,3,5,10]
#     RiskThresholds=[0.5,0.6,0.7,0.8]
#     timestepslist=[20,50,100,250,500,1000]
    QualityThreshold=0.5
    mc_num_steps = 100
    mc = True
    configlist = ["CHO","DHO","HHO","CHE","DHE","HHE"]
    test_table = pd.DataFrame()
    for config in configlist:
        for risk in RiskThresholds:
            RiskThreshold = risk
            for timestep in timestepslist:
                timesteps = timestep
                for numTasks in numTasksBeforeFaillist:
                    numTasksBeforeFail = numTasks
                    try:
                        if mc:
                            print(f'\nMC {config}, Risk:{RiskThreshold}, t:{timesteps}, #Tasks Before Fail:{numTasksBeforeFail}')
                            mcdf = SRRSmc(config,timesteps,mc_num_steps,False)
                            mean_mcdf = mcdf[['Print Capacity', 'Assembling Capacity', 'Collection Capacity', 'MTBF','MTTR', 'MDT', 'Aoss','#In', '#Out','Average Build Quality of System']].mean()
                            plotDistribution(config,mcdf)
                        else:
                            print(f'Deterministic {config}, Risk:{RiskThreshold}, t:{timesteps}, #Tasks Before Fail:{numTasksBeforeFail}')
                            df,ram_df = SRRS(config,timesteps,False)
                            result = df.tail(1).reset_index(drop=True)
                            result = pd.concat([result,ram_df], axis=1)
                            result['RiskThreshold'] = RiskThreshold
                            result['Configuration'] = config
                            result = result[['Configuration','#Tasks before failure','RiskThreshold','Time', 'NonPr', 'Printable', 'Materials', 'Env_Materials',
                           '#Replicator', '#Normal', '#Assembler', '#Printer', '#Assembling',
                           '#Printing', '#Collecting', '#Idle', '#Repair', '#In', '#Out',
                           'Average Build Quality in-service', 'Average Build Quality of System',
                           '#WasteReplicator', '#WasteNormal', '#WasteAssembler', '#WastePrinter',
                           'Environment Exhaust Time', 'Printable Exhaust Time',
                           'NonPr Exhaust Time', 'Material Exhaust Time', 'Average Risk',
                           'Print Capacity', 'Assembling Capacity', 'Collection Capacity',
                           'MTBF', 'MTTR', 'MDT', 'Aoss']]
                            test_table = pd.concat([test_table,result],ignore_index=True)
                    except Exception as err:
                        print(f"Unexpected {err=}, {type(err)=}")
                        raise
    if mc:
        pass
    else:
#         display(test_table)
        filename = "deterministic-scenarios.csv"
        print(f'saving {filename}...')
        test_table.to_csv(filename)
    

if __name__ == "__main__":
    main()


MC CHO, Risk:0.5, t:500, #Tasks Before Fail:2
Running iteration: 100

,Time,NonPr,Printable,Materials,Env_Materials,#Replicator,#Normal,#Assembler,#Printer,#Assembling,#Printing,#Collecting,#Idle,#Repair,#In,#Out,Average Build Quality in-service,Average Build Quality of System,#WasteReplicator,#WasteNormal,#WasteAssembler,#WastePrinter,Environment Exhaust Time,Printable Exhaust Time,NonPr Exhaust Time,Material Exhaust Time,Average Risk,Print Capacity,Assembling Capacity,Collection Capacity,MTBF,MTTR,MDT,Aoss,#Tasks before failure
0,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.76,0.76,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.110592,1.0,1.0,117.0,41535.258591,0.080451,10.0,0.999759,2
1,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.769,0.769,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.106909,1.0,1.0,117.0,1561.058635,0.13253,5.0,0.996807,2
2,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.734,0.734,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.111552,1.0,1.0,117.0,1545.081717,0.083746,10.0,0.993569,2
3,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.815,0.815,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.105826,1.0,1.0,117.0,1395.006404,0.132858,5.0,0.996429,2
4,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.805,0.805,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.106159,1.0,1.0,117.0,4062.977463,0.13055,5.0,0.998771,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.721,0.721,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.112352,1.0,1.0,117.0,7670715.712007,0.061128,15.0,0.999998,2
96,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.788,0.788,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.105742,1.0,1.0,117.0,14.913444,0.176607,3.0,0.832528,2
97,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.764,0.764,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.109872,1.0,1.0,117.0,669.407797,0.08824,9.0,0.986734,2
98,499.0,184.0,2.0,416.0,0.0,1.0,116.0,0.0,0.0,0.0,1.0,0.0,116.0,0.0,117.0,0.0,0.785,0.785,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.108576,1.0,1.0,117.0,48.428095,0.111503,7.0,0.87371,2



MC CHO, Risk:0.75, t:500, #Tasks Before Fail:2
Running iteration: 2

/var/folders/fl/s4bc006j105fpvzv3f3tbnyw0000gn/T/ipykernel_3369/3697143361.py:145: RuntimeWarning:

invalid value encountered in long_scalars

/var/folders/fl/s4bc006j105fpvzv3f3tbnyw0000gn/T/ipykernel_3369/3697143361.py:145: RuntimeWarning:

invalid value encountered in long_scalars



Running iteration: 4

...


MC DHE, Risk:0.75, t:500, #Tasks Before Fail:2
Running iteration: 100

,Time,NonPr,Printable,Materials,Env_Materials,#Replicator,#Normal,#Assembler,#Printer,#Assembling,#Printing,#Collecting,#Idle,#Repair,#In,#Out,Average Build Quality in-service,Average Build Quality of System,#WasteReplicator,#WasteNormal,#WasteAssembler,#WastePrinter,Environment Exhaust Time,Printable Exhaust Time,NonPr Exhaust Time,Material Exhaust Time,Average Risk,Print Capacity,Assembling Capacity,Collection Capacity,MTBF,MTTR,MDT,Aoss,#Tasks before failure
0,499.0,0.0,49.0,1.0,0.0,0.0,0.0,51.0,57.0,0.0,0.0,0.0,108.0,0.0,108.0,39.0,0.699,0.622,0.0,0.0,20.0,19.0,499.0,0.0,499.0,499.0,0.10471,57.0,51.0,108.0,154.209869,0.088897,4.646037,0.970753,2
1,499.0,0.0,49.0,1.0,0.0,0.0,0.0,56.0,62.0,0.0,0.0,0.0,118.0,0.0,118.0,30.0,0.663,0.617,0.0,0.0,14.0,16.0,499.0,0.0,499.0,499.0,0.104666,62.0,56.0,118.0,1271.328506,0.056622,6.294017,0.995074,2
2,499.0,0.0,49.0,1.0,0.0,0.0,0.0,57.0,68.0,0.0,0.0,0.0,125.0,0.0,125.0,26.0,0.709,0.658,0.0,0.0,12.0,14.0,499.0,0.0,499.0,499.0,0.104737,68.0,57.0,125.0,1917.546503,0.059851,6.610256,0.996565,2
3,499.0,0.0,49.0,1.0,0.0,0.0,0.0,48.0,52.0,0.0,0.0,0.0,100.0,0.0,100.0,47.0,0.657,0.583,0.0,0.0,15.0,32.0,499.0,0.0,499.0,499.0,0.121025,52.0,48.0,100.0,0.098877,0.078461,4.038459,0.023899,2
4,499.0,0.0,49.0,1.0,0.0,0.0,0.0,57.0,58.0,0.0,0.0,0.0,115.0,0.0,115.0,31.0,0.638,0.591,0.0,0.0,19.0,12.0,499.0,0.0,499.0,499.0,0.111934,58.0,57.0,115.0,8455.859132,0.060676,4.991484,0.99941,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,499.0,0.0,49.0,1.0,0.0,0.0,0.0,50.0,43.0,0.0,0.0,0.0,93.0,0.0,93.0,53.0,0.637,0.555,0.0,0.0,24.0,29.0,499.0,0.0,499.0,499.0,0.118727,43.0,50.0,93.0,2314617.932371,0.089825,4.343168,0.999998,2
96,499.0,0.0,49.0,1.0,0.0,0.0,0.0,51.0,54.0,0.0,0.0,0.0,105.0,0.0,105.0,47.0,0.658,0.578,0.0,0.0,18.0,29.0,499.0,0.0,499.0,499.0,0.110417,54.0,51.0,105.0,3.068182,0.081522,3.78355,0.447797,2
97,499.0,0.0,49.0,1.0,0.0,0.0,0.0,44.0,53.0,0.0,0.0,0.0,97.0,0.0,97.0,52.0,0.63,0.56,0.0,0.0,24.0,28.0,499.0,0.0,499.0,499.0,0.127081,53.0,44.0,97.0,12555.363992,0.105244,3.400575,0.999729,2
98,499.0,0.0,49.0,1.0,0.0,0.0,0.0,47.0,50.0,0.0,0.0,0.0,97.0,0.0,97.0,51.0,0.679,0.588,0.0,0.0,26.0,25.0,499.0,0.0,499.0,499.0,0.124278,50.0,47.0,97.0,1827.562705,0.07804,4.156289,0.997731,2



MC HHE, Risk:0.5, t:500, #Tasks Before Fail:2
Running iteration: 100

,Time,NonPr,Printable,Materials,Env_Materials,#Replicator,#Normal,#Assembler,#Printer,#Assembling,#Printing,#Collecting,#Idle,#Repair,#In,#Out,Average Build Quality in-service,Average Build Quality of System,#WasteReplicator,#WasteNormal,#WasteAssembler,#WastePrinter,Environment Exhaust Time,Printable Exhaust Time,NonPr Exhaust Time,Material Exhaust Time,Average Risk,Print Capacity,Assembling Capacity,Collection Capacity,MTBF,MTTR,MDT,Aoss,#Tasks before failure
0,499.0,0.0,49.0,1.0,0.0,0.0,53.0,34.0,66.0,0.0,0.0,0.0,153.0,0.0,153.0,18.0,0.697,0.67,0.0,10.0,3.0,5.0,499.0,0.0,499.0,499.0,0.140998,66.0,34.0,153.0,680213.590846,0.051798,14.616883,0.999979,2
1,499.0,0.0,49.0,1.0,0.0,0.0,40.0,25.0,50.0,0.0,0.0,0.0,115.0,0.0,115.0,32.0,0.675,0.624,0.0,10.0,9.0,13.0,499.0,0.0,499.0,499.0,0.17517,50.0,25.0,115.0,1947675.427699,0.062675,14.59704,0.999993,2
2,499.0,0.0,49.0,1.0,0.0,0.0,41.0,37.0,47.0,0.0,0.0,0.0,125.0,0.0,125.0,31.0,0.693,0.642,0.0,12.0,6.0,13.0,499.0,0.0,499.0,499.0,0.14308,47.0,37.0,125.0,34111993.814547,0.053407,14.851389,1.0,2
3,499.0,0.0,49.0,1.0,0.0,0.0,53.0,31.0,66.0,0.0,0.0,0.0,150.0,0.0,150.0,15.0,0.713,0.688,0.0,5.0,7.0,3.0,499.0,0.0,499.0,499.0,0.136518,66.0,31.0,150.0,38402193.456396,0.059223,14.600794,1.0,2
4,499.0,0.0,49.0,1.0,0.0,0.0,32.0,24.0,43.0,0.0,0.0,0.0,99.0,0.0,99.0,43.0,0.659,0.585,0.0,15.0,8.0,20.0,499.0,0.0,499.0,499.0,0.195672,43.0,24.0,99.0,3825390.156662,0.05487,17.138428,0.999996,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,499.0,0.0,49.0,1.0,0.0,0.0,47.0,36.0,68.0,0.0,0.0,0.0,151.0,0.0,151.0,18.0,0.674,0.65,0.0,6.0,3.0,9.0,499.0,0.0,499.0,499.0,0.130624,68.0,36.0,151.0,1173767069.163734,0.040492,22.050794,1.0,2
96,499.0,0.0,49.0,1.0,0.0,0.0,42.0,31.0,61.0,0.0,0.0,0.0,134.0,0.0,134.0,18.0,0.682,0.655,0.0,5.0,1.0,12.0,499.0,0.0,499.0,499.0,0.169005,61.0,31.0,134.0,349276727.893582,0.047599,17.442857,1.0,2
97,499.0,0.0,49.0,1.0,0.0,0.0,46.0,33.0,55.0,0.0,0.0,0.0,134.0,0.0,134.0,20.0,0.701,0.668,0.0,4.0,6.0,10.0,499.0,0.0,499.0,499.0,0.15828,55.0,33.0,134.0,4058883.654398,0.061321,12.443733,0.999997,2
98,499.0,0.0,49.0,1.0,0.0,0.0,37.0,30.0,47.0,0.0,0.0,0.0,114.0,0.0,114.0,34.0,0.671,0.613,0.0,10.0,6.0,18.0,499.0,0.0,499.0,499.0,0.168753,47.0,30.0,114.0,5062210.106577,0.049034,17.747494,0.999996,2



MC HHE, Risk:0.75, t:500, #Tasks Before Fail:2
Running iteration: 39